## Instruction

In this notebook, I will download and process stock data like P4 project. It is preparing for backtesting.

In [1]:
import pandas as pd
import numpy as np
import time
from tqdm import tqdm

import tushare as ts

In [2]:
# register token
token = '' # your token
ts.set_token(token)
pro = ts.pro_api()

## Pick Stock List
We got stock list by each day, and pick up stock exist in that day with some ohter custom constraints.

In [3]:
# trade calendar
start_date, end_date = '20230101' , '20230327'
calendar = pro.trade_cal(exchange='SSE', is_open='1', 
                            start_date=start_date, 
                            end_date=end_date, 
                            fields='cal_date')
calendar = calendar.values.flatten()[::-1]
print((calendar[0], calendar[-1]), len(calendar))

('20230103', '20230327') 55


In [4]:
# get stock list info
def get_basic_from_Tushare(trade_date):
    # column: list_date(exist on )
    basic_fields = 'trade_date,ts_code,name,industry,list_date'
    df = pro.bak_basic(trade_date=trade_date, fields=basic_fields)
    return df
# test function
df = get_basic_from_Tushare(calendar[0])
df.head()

,trade_date,ts_code,name,industry,list_date
0,20230103,000001.SZ,平安银行,银行,19910403
1,20230103,000002.SZ,万 科Ａ,全国地产,19910129
2,20230103,000004.SZ,ST国华,软件服务,19910114
3,20230103,000005.SZ,ST星源,环境保护,19901210
4,20230103,000006.SZ,深振业Ａ,区域地产,19920427


In [5]:
# filter stock df
my_indusry = {'医药商业':1,'中成药':2,'生物制药':3,'化学制药':4,'互联网':5,'软件服务':6}
def filter_basic_stockDf(df):
    # got sotck exist on that day
    df = df.loc[df.list_date!='0']
    # filter custom industry
    df = df.loc[df.industry.isin(my_indusry.keys())]
    return df
# test function
df = filter_basic_stockDf(df)
print(df.shape)
df.head()

(668, 5)


,trade_date,ts_code,name,industry,list_date
2,20230103,000004.SZ,ST国华,软件服务,19910114
21,20230103,000028.SZ,国药一致,医药商业,19930809
26,20230103,000034.SZ,神州数码,软件服务,19940509
52,20230103,000078.SZ,海王生物,医药商业,19981218
59,20230103,000150.SZ,*ST宜康,生物制药,20000807


## Get Fundamental df from csv (Option)

In [6]:
fundamental_df = pd.read_csv('fundamental_20170405_20230317.csv').iloc[:,1:]
fundamental_df.head()

,trade_date,ts_code,name,industry,list_date
0,20170103,300573.SZ,兴齐眼药,化学制药,20161208.0
1,20170103,300571.SZ,平治信息,互联网,20161213.0
2,20170103,300579.SZ,数字认证,软件服务,20161223.0
3,20170103,300561.SZ,汇金科技,软件服务,20161117.0
4,20170103,603716.SH,塞力斯,医药商业,20161031.0


## Download Data by Tickers

In [7]:
def get_daydata_from_Tushare(fundamental_df):
    
    def get_daydata_by_ticker_(ts_code):
        # indicator check: https://tushare.pro/document/2?doc_id=32
        my_fields = 'ts_code,trade_date,turnover_rate,volume_ratio,pe,pb,total_share,free_share,total_mv,circ_mv'
        df = pro.query('daily_basic', ts_code=ts_code, start_date=calendar[0], end_date=calendar[-1], fields=my_fields)
        return df
    
    tickers = fundamental_df['ts_code'].values
    df = pd.DataFrame()
    for ts_code in tickers:
        ticker_df = get_daydata_by_ticker_(ts_code)
        tmp = fundamental_df.loc[fundamental_df.ts_code==ts_code]
        ticker_df['name'] = tmp['name'].unique()[0]
        ticker_df['industry'] = tmp['industry'].unique()[0]
        ticker_df['list_date'] = tmp['list_date'].unique()[0]
        df = df.append(ticker_df)
    return df
    
# test function 
df_20170103 = get_daydata_from_Tushare(df.iloc[:2,])
print(df_20170103.shape)
df_20170103.head()

(110, 13)


,ts_code,trade_date,turnover_rate,volume_ratio,pe,pb,total_share,free_share,total_mv,circ_mv,name,industry,list_date
0,000004.SZ,20230327,2.2825,1.79,NaN,1.4956,13284.8008,8650.0178,133246.5520,119273.6036,ST国华,软件服务,19910114
1,000004.SZ,20230324,2.3253,2.37,NaN,1.5225,13284.8008,8650.0178,135637.8162,121414.1069,ST国华,软件服务,19910114
2,000004.SZ,20230323,2.0266,2.67,NaN,1.5046,13284.8008,8650.0178,134043.6401,119987.1047,ST国华,软件服务,19910114
3,000004.SZ,20230322,0.6369,0.82,NaN,1.4643,13284.8008,8650.0178,130456.7439,116776.3496,ST国华,软件服务,19910114
4,000004.SZ,20230321,0.6752,0.74,NaN,1.4584,13284.8008,8650.0178,129925.3518,116300.6822,ST国华,软件服务,19910114


In [8]:
def get_dayIndicator_from_Tushare(fundamental_df):
    
    def get_daydata_by_ticker_(ts_code):
        # inidcator check: https://tushare.pro/document/2?doc_id=296
        my_fields = 'ts_code,trade_date,close_qfq,open_qfq,high_qfq,low_qfq,amount,vol,cci'
        df = pro.stk_factor(ts_code=ts_code, start_date=calendar[0], end_date=calendar[-1], fields=my_fields)
        df.rename(columns = {'close_qfq':'close', 'open_qfq':'open', 'high_qfq':'high', 'low_qfq':'low'}, inplace = True)
        return df
    
    tickers = fundamental_df['ts_code'].values
    df = pd.DataFrame()
    for ts_code in tickers:
        ticker_df = get_daydata_by_ticker_(ts_code)
        df = df.append(ticker_df)
    return df
        
df_20170103_ = get_dayIndicator_from_Tushare(df.iloc[:2,])
df_20170103 = df_20170103.merge(df_20170103_, on=['trade_date','ts_code'], how='left')
print(df_20170103.shape)
df_20170103.head()

(110, 19)


,ts_code,trade_date,turnover_rate,volume_ratio,pe,pb,total_share,free_share,total_mv,circ_mv,name,industry,list_date,amount,open,close,high,low,cci
0,000004.SZ,20230327,2.2825,1.79,NaN,1.4956,13284.8008,8650.0178,133246.5520,119273.6036,ST国华,软件服务,19910114,27508.11,10.35,10.03,10.48,9.96,108.261
1,000004.SZ,20230324,2.3253,2.37,NaN,1.5225,13284.8008,8650.0178,135637.8162,121414.1069,ST国华,软件服务,19910114,28306.69,10.10,10.21,10.37,10.10,168.292
2,000004.SZ,20230323,2.0266,2.67,NaN,1.5046,13284.8008,8650.0178,134043.6401,119987.1047,ST国华,软件服务,19910114,24218.87,9.84,10.09,10.18,9.84,116.043
3,000004.SZ,20230322,0.6369,0.82,NaN,1.4643,13284.8008,8650.0178,130456.7439,116776.3496,ST国华,软件服务,19910114,7443.02,9.77,9.82,9.87,9.76,-25.006
4,000004.SZ,20230321,0.6752,0.74,NaN,1.4584,13284.8008,8650.0178,129925.3518,116300.6822,ST国华,软件服务,19910114,7825.35,9.66,9.78,9.81,9.64,-80.226


## Download ALL

In [10]:
fundamental_df1 = get_basic_from_Tushare(calendar[0])
#fundamental_df1.tail()

In [13]:
# download start day
win_len = 5
#fundamental_df = get_basic_from_Tushare(calendar[0])
#fundamental_df = filter_basic_stockDf(df)

# if use fundamental by file excute below
fundamental_df = fundamental_df1.loc[fundamental_df1.ts_code.isin(fundamental_df.ts_code)]

universe_raw = pd.DataFrame()
for i in tqdm(range(0,fundamental_df.shape[0]-1,win_len), desc='downloaing'):
    tmp = fundamental_df.iloc[i:i+win_len,]
    universe = get_daydata_from_Tushare(tmp)
    tmp = get_dayIndicator_from_Tushare(tmp)
    universe = universe.merge(tmp, on=['trade_date','ts_code'], how='left')
    universe_raw = universe_raw.append(universe)

if (i+win_len) <= (fundamental_df.shape[0]-1):
    tmp = fundamental_df.iloc[i+win_len:]
    universe = get_daydata_from_Tushare(tmp)
    tmp = get_dayIndicator_from_Tushare(tmp)
    universe = universe.merge(tmp, on=['trade_date','ts_code'], how='left')
    universe_raw = universe_raw.append(universe)
    
print(universe_raw.shape)
universe_raw.head()

downloaing: 100%|███████████████████████████████| 94/94 [02:22<00:00,  1.52s/it]

(25726, 19)


,ts_code,trade_date,turnover_rate,volume_ratio,pe,pb,total_share,free_share,total_mv,circ_mv,name,industry,list_date,amount,open,close,high,low,cci
0,000028.SZ,20230327,1.4743,0.79,13.9609,1.157,42812.6983,12791.6361,1.865777e+06,1.602583e+06,国药一致,医药商业,19930809,238210.32,43.80,43.58,44.70,43.20,-17.209
1,000028.SZ,20230324,1.9247,0.75,14.2236,1.1788,42812.6983,12791.6361,1.900884e+06,1.632737e+06,国药一致,医药商业,19930809,315702.69,46.00,44.40,46.00,44.00,22.654
2,000028.SZ,20230323,1.0107,0.33,14.5792,1.2082,42812.6983,12791.6361,1.948406e+06,1.673556e+06,国药一致,医药商业,19930809,169603.38,45.18,45.51,46.10,45.00,53.448
3,000028.SZ,20230322,1.5294,0.51,14.576,1.208,42812.6983,12791.6361,1.947978e+06,1.673188e+06,国药一致,医药商业,19930809,257025.84,45.20,45.50,46.62,44.50,65.694
4,000028.SZ,20230321,1.6397,0.55,14.6401,1.2133,42812.6983,12791.6361,1.956540e+06,1.680543e+06,国药一致,医药商业,19930809,277642.69,46.41,45.70,47.08,45.38,93.991


In [14]:
# download other day
for trade_date in tqdm(calendar[1:], desc='funmental_df downloading'):
    df = get_basic_from_Tushare(trade_date)
    df = filter_basic_stockDf(df)
    df = df.loc[df.ts_code.isin(fundamental_df.ts_code)==False]
    fundamental_df = fundamental_df.append(df)
    if df.empty != True:
        universe = get_daydata_from_Tushare(df)
        tmp = get_dayIndicator_from_Tushare(df)
        universe = universe.merge(tmp, on=['trade_date','ts_code'], how='left')
        universe_raw = universe_raw.append(universe)
        
print(universe_raw.shape)
universe_raw.head()

funmental_df downloading: 100%|█████████████████| 54/54 [01:47<00:00,  1.98s/it]

(39927, 19)


,ts_code,trade_date,turnover_rate,volume_ratio,pe,pb,total_share,free_share,total_mv,circ_mv,name,industry,list_date,amount,open,close,high,low,cci
0,000028.SZ,20230327,1.4743,0.79,13.9609,1.157,42812.6983,12791.6361,1.865777e+06,1.602583e+06,国药一致,医药商业,19930809,238210.32,43.80,43.58,44.70,43.20,-17.209
1,000028.SZ,20230324,1.9247,0.75,14.2236,1.1788,42812.6983,12791.6361,1.900884e+06,1.632737e+06,国药一致,医药商业,19930809,315702.69,46.00,44.40,46.00,44.00,22.654
2,000028.SZ,20230323,1.0107,0.33,14.5792,1.2082,42812.6983,12791.6361,1.948406e+06,1.673556e+06,国药一致,医药商业,19930809,169603.38,45.18,45.51,46.10,45.00,53.448
3,000028.SZ,20230322,1.5294,0.51,14.576,1.208,42812.6983,12791.6361,1.947978e+06,1.673188e+06,国药一致,医药商业,19930809,257025.84,45.20,45.50,46.62,44.50,65.694
4,000028.SZ,20230321,1.6397,0.55,14.6401,1.2133,42812.6983,12791.6361,1.956540e+06,1.680543e+06,国药一致,医药商业,19930809,277642.69,46.41,45.70,47.08,45.38,93.991


## Clean Data

In [16]:
# check exist stock list
data_L = pro.query('stock_basic',list_status='L', 
                   fields='ts_code,symbol,name,area,industry,list_date,delist_date,market')
data_L = data_L.loc[data_L.industry.isin(my_indusry)]
#data_L

In [17]:
# remove close is null
universe_raw = universe_raw.loc[universe_raw.close.isnull()==False]
fundamental_df = fundamental_df.loc[fundamental_df.ts_code.isin(universe_raw.ts_code)]

In [18]:
# remove amount > 0.5 million yuan
universe_raw = universe_raw.loc[universe_raw['amount']>=500]

In [19]:
# removw ST
def column_from_names(n, name):
    return list(filter(lambda x: name in x, n))
ST_list = column_from_names(universe_raw['name'].unique(), 'ST')
universe_raw = universe_raw.loc[universe_raw.name.isin(ST_list)==False]

In [20]:
# update fundamental df
fundamental_df = fundamental_df.loc[fundamental_df.ts_code.isin(universe_raw.ts_code.unique())]

In [21]:
# reset pe is null to 1.e3
universe_raw['pe'] = np.where(universe_raw['pe'].isnull(), 1.*1e3, universe_raw['pe'])
# fill pb null with a large value
universe_raw['pb'] = np.where(universe_raw['pb'].isnull(), 1.*1e2, universe_raw['pb'])

## Download Factor from Tushare

In [22]:
def find_next_close_date(date):
    if date in calendar:
        return date
    int_date = int(date)
    min_interval = np.inf
    idx = 0
    for ii,dt in enumerate(calendar):
        if int(dt) > int_date and int(dt) - int_date < min_interval:
            min_interval = int(dt) - int_date
            idx = ii
    return calendar[idx]
        
# download profit notice
def get_profit_notice_from_Tushare(ts_code):
    start_date='20161201'
    my_fields = 'ts_code,ann_date,first_ann_date,type,p_change_min,p_change_max'
    df = pro.forecast_vip(ts_code=ts_code ,start_date=start_date, end_date=calendar[-1] ,fields=my_fields)
    df['first_ann_date'] = np.where(df['first_ann_date'].isnull(), df['ann_date'], df['first_ann_date'])
    df.fillna(method='bfill', inplace=True)
    return df

# test function
get_profit_notice_from_Tushare('603538.SH')

,ts_code,ann_date,type,p_change_min,p_change_max,first_ann_date
0,603538.SH,20230131,预增,126.76,166.78,20230131
1,603538.SH,20220705,预增,54.70,71.89,20220705
2,603538.SH,20200122,预增,45.00,60.00,20200122
3,603538.SH,20190122,预增,116.00,131.00,20190122
4,603538.SH,20171018,略减,-45.00,-35.00,20171018
5,603538.SH,20170316,略增,5.00,20.00,20170316


In [18]:
# download all profit notice
ticker_list = fundamental_df.ts_code.unique()
notice_df_all = pd.DataFrame()
for ts_code in tqdm(ticker_list, desc='download profit notice'):
    notice_df = get_profit_notice_from_Tushare(ts_code)
    for ii, dt in enumerate(notice_df['first_ann_date']):
        dt = find_next_close_date(dt)
        notice_df.at[ii,'first_ann_date'] = dt
    notice_df['trade_date'] = notice_df['first_ann_date'].apply(np.int64)
    notice_df = notice_df.drop_duplicates(subset=['trade_date'])
    notice_df_all = notice_df_all.append(notice_df)
    
universe = universe_raw.copy(deep=True)
universe = universe.merge(notice_df_all[['ts_code','trade_date','type','p_change_min','p_change_max']], 
                        on=['ts_code','trade_date'], how='left')

download profit notice: 100%|█████████████████| 651/651 [01:29<00:00,  7.26it/s]


In [19]:
# fillna use ffill then fillna(0)
universe_raw = pd.DataFrame()
for group_ticker in tqdm(universe.groupby('ts_code'), desc='aggregeate df'):
    ticker = group_ticker[1]
    ticker[['type', 'p_change_min', 'p_change_max']] = ticker[['type', 'p_change_min', 'p_change_max']].fillna(method='ffill')
    ticker['type'].fillna('不确定',inplace=True)
    ticker[['p_change_min', 'p_change_max']] = ticker[['p_change_min', 'p_change_max']].fillna(0.)
    universe_raw = universe_raw.append(ticker)

aggregeate df: 100%|█████████████████████████| 651/651 [00:05<00:00, 111.02it/s]


In [20]:
# add type value
universe_raw['type_value'] = 0
type_dict = {'不确定':0, '预增':2, '首亏':-2, '预减':-2, '扭亏':0, '续亏':-3, '略增':1, '续盈':3, '略减':-1}
for key in type_dict.keys():
    universe_raw['type_value'] = np.where(universe_raw['type']==key, type_dict[key], universe_raw['type_value'])

## Download IPO info 

In [21]:
df1 = pro.new_share(start_date='20140101', end_date='20200101')
time.sleep(1)
df2 = pro.new_share(start_date='20200102', end_date='20230318')
df = df1.append(df2)
print(df.shape)
df.head()

(2765, 12)


,ts_code,sub_code,name,ipo_date,issue_date,amount,market_amount,price,pe,limit_amount,funds,ballot
0,002973.SZ,002973,侨银股份,20191225,20200106,4089.0,3680.0,5.74,22.99,1.2,2.347,0.04
1,688181.SH,787181,八亿时空,20191225,20200106,2412.0,828.0,43.98,37.35,0.6,3.532,0.04
2,300812.SZ,300812,易天股份,20191225,20200109,1938.0,1938.0,21.46,22.99,1.9,4.159,0.02
3,688081.SH,787081,兴图新科,20191224,20200106,1840.0,699.0,28.21,51.70,0.5,4.558,0.04
4,300811.SZ,300811,铂科新材,20191219,20191230,1440.0,1440.0,26.22,22.99,1.4,3.776,0.01


In [22]:
universe = pd.DataFrame()
for group_ticker in tqdm(universe_raw.groupby('ts_code'), desc='aggregeate df'):
    ticker = group_ticker[1]
    if group_ticker[0] in df.ts_code.values:
        tmp = df.loc[df.ts_code==group_ticker[0]]
        ticker['list_date'] = tmp['issue_date'].values[0]
        ticker['issue_price'] = tmp['price'].values[0]
        ticker['issue_amount'] = tmp['amount'].values[0] # issue stcke unite is 10000 share
    else:
        ticker['issue_price'] = ticker.iloc[0,:]['open']
        ticker['issue_amount'] = ticker.iloc[0,:]['free_share']
    universe = universe.append(ticker)
universe_raw = universe      
#ticker[['trade_date','ts_code','name','list_date','issue_price','issue_amount','free_share']]

aggregeate df: 100%|█████████████████████████| 651/651 [00:04<00:00, 130.22it/s]


## Download Finance Info

In [23]:
def get_finace_reports_from_Tushre(ts_code):
    # https://tushare.pro/document/2?doc_id=79
    start_date='20161201'
    my_fields = [
                  'ts_code','ann_date','cfps','revenue_ps', 'quick_ratio', # 每股现金流，每股营业收入，速冻比率
                  'dt_eps','basic_eps_yoy','dt_eps_yoy', # 每股收益
                  'bps','bps_yoy', # 每股净资产
                  'extra_item','profit_dedt', # 扣非，扣非净利润，扣费净利润单季度 
                  'roe_dt','q_dt_roe', # 净资产收益
                  'capital_rese_ps','surplus_rese_ps', # 每股资本公积，每股公积盈余 
                  'gross_margin','interestdebt','ca_to_assets',# 毛利，速冻比率，带息债务
                  'ebt_yoy','roe_yoy','or_yoy','equity_yoy' # 总利润增长，净资产收益增长，营业收入增长，净资产增长
                 ]
    df = pro.fina_indicator(ts_code=ts_code, start_date=start_date, end_date=calendar[-1])[my_fields]
    df = df.fillna(method='bfill').dropna()
    return df

# function test
get_finace_reports_from_Tushre('603538.SH').head()

,ts_code,ann_date,cfps,revenue_ps,quick_ratio,dt_eps,basic_eps_yoy,dt_eps_yoy,bps,bps_yoy,...,roe_yoy,capital_rese_ps,surplus_rese_ps,gross_margin,interestdebt,ca_to_assets,ebt_yoy,roe_yoy,or_yoy,equity_yoy
0,603538.SH,20221028,0.1674,5.7783,0.8542,0.95,54.5455,53.2258,8.8382,-20.2890,...,48.4919,2.5581,0.1163,5.027804e+08,1.424953e+09,43.3206,60.0877,48.4919,33.9594,6.4431
1,603538.SH,20220816,0.2172,4.2131,0.9240,0.82,52.6316,54.7170,8.6753,-21.7581,...,49.0908,2.5336,0.1163,3.791913e+08,1.409538e+09,43.8655,66.9877,49.0908,29.7380,6.4495
2,603538.SH,20220426,0.0479,3.2814,1.1100,0.70,123.5294,105.8824,12.0966,9.0983,...,106.9220,4.1745,0.1626,2.025354e+08,1.295483e+09,42.7028,130.5781,106.9220,47.8641,10.6430
3,603538.SH,20220426,1.3101,8.3147,0.9790,0.92,-9.4340,-13.2075,11.0878,6.9033,...,-18.3608,3.8807,0.1638,4.692199e+08,1.260414e+09,42.5440,-12.0170,-18.3608,5.4278,11.9375
4,603538.SH,20211028,0.4900,6.1415,1.1275,0.87,-5.1546,-9.3750,11.2975,8.9252,...,-20.2690,3.7641,0.1656,3.392502e+08,1.148648e+09,43.1607,-7.8135,-20.2690,-3.1838,20.4111


In [24]:
# download all profit notice
ticker_list = fundamental_df.ts_code.unique()
finance_df_all = pd.DataFrame()
for ts_code in tqdm(ticker_list, desc='finance info download...'):
    finance_df = get_finace_reports_from_Tushre(ts_code)
    for ii, dt in enumerate(finance_df['ann_date']):
        dt = find_next_close_date(dt)
        finance_df.at[ii,'ann_date'] = dt
    finance_df['trade_date'] = finance_df['ann_date'].apply(np.int64)
    finance_df = finance_df.drop_duplicates(subset=['trade_date']).drop(columns=['ann_date'])
    finance_df_all = finance_df_all.append(finance_df)

print(finance_df_all.shape)
finance_df_all.head()

finance info download...: 100%|███████████████| 651/651 [02:00<00:00,  5.40it/s]

(674, 24)


,ts_code,cfps,revenue_ps,quick_ratio,dt_eps,basic_eps_yoy,dt_eps_yoy,bps,bps_yoy,extra_item,...,capital_rese_ps,surplus_rese_ps,gross_margin,interestdebt,ca_to_assets,ebt_yoy,roe_yoy,or_yoy,equity_yoy,trade_date
0,000028.SZ,2.3335,127.5714,1.0788,2.4500,-2.3904,-2.3904,36.7084,5.2993,3.935494e+07,...,10.2115,0.5000,6.273907e+09,5.167881e+09,76.9456,-1.8966,-8.9000,8.7766,7.0603,20230103
0,000034.SZ,0.0428,127.2957,0.6798,1.0610,1297.7632,1301.5852,10.5968,15.1401,2.156349e+07,...,5.7927,0.1977,3.345509e+09,1.284529e+10,80.0224,326.5626,1147.1870,-1.7574,19.3695,20230103
0,000078.SZ,-0.2254,10.5262,0.9462,0.0480,-38.2239,-38.2239,2.1606,2.0836,1.900036e+07,...,0.8393,0.0190,2.968007e+09,1.205603e+10,84.0299,-36.9918,-38.4710,-5.9134,0.3232,20230103
0,000153.SZ,0.3417,9.1801,0.6284,0.4053,40.2422,40.2422,5.4893,12.1249,3.915050e+07,...,1.8005,0.1251,7.986624e+08,5.703331e+08,46.0225,39.4221,23.0594,9.3871,13.9467,20230103
0,000158.SZ,-0.3988,3.6553,0.9643,-0.0848,-303.8095,-303.8095,3.7647,-2.2029,2.027148e+08,...,1.6642,0.1613,4.748181e+08,5.369378e+09,61.6890,-135.0122,-302.1903,-4.8610,0.5674,20230103


In [28]:
bak_data = universe.copy(deep=True)
print(len(universe.ts_code.unique()), len(finance_df_all.loc[finance_df_all['trade_date']>20221231].ts_code.unique()))

651 628


In [41]:
#finance_df_all = finance_df_all.loc[:, ~finance_df_all.columns.duplicated()]


In [44]:
# merage recent
fin_field = ['cfps', 'revenue_ps', 'quick_ratio', 'dt_eps',
       'basic_eps_yoy', 'dt_eps_yoy', 'bps', 'bps_yoy', 'extra_item',
       'profit_dedt', 'roe_dt', 'q_dt_roe', 'capital_rese_ps',
       'surplus_rese_ps', 'gross_margin', 'interestdebt', 'ca_to_assets',
       'ebt_yoy', 'roe_yoy', 'or_yoy', 'equity_yoy']

universe.drop_duplicates(['trade_date','ts_code'],inplace=True)
finance_df_all.drop_duplicates(['trade_date','ts_code'],inplace=True)
universe_raw = pd.DataFrame()
for ts_code in tqdm(universe.ts_code.unique()):
    tmp = universe.loc[universe.ts_code==ts_code]
    fin_tmp = finance_df_all.loc[finance_df_all.ts_code==ts_code]
    if fin_tmp.empty:
        print(f'found empty {ts_code}')
        continue
    fin_tmp = fin_tmp.iloc[0,:]
    for col in fin_field:
        tmp[col] = fin_tmp[col]
    universe_raw = universe_raw.append(tmp, ignore_index=True)
universe_raw.head()    

  0%|                                                   | 0/651 [00:00<?, ?it/s]/Users/jiaruiming/anaconda/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 63%|█████████████████████████▉               | 411/651 [00:07<00:04, 50.29it/s]

found empty 301378.SZ
found empty 600085.SH


 74%|██████████████████████████████▍          | 483/651 [00:09<00:03, 48.23it/s]

found empty 600829.SH


 85%|██████████████████████████████████▉      | 555/651 [00:11<00:02, 41.00it/s]

...

found empty 688428.SH


 98%|████████████████████████████████████████▎| 641/651 [00:13<00:00, 36.66it/s]

found empty 688561.SH


100%|█████████████████████████████████████████| 651/651 [00:13<00:00, 48.81it/s]


,ts_code,trade_date,turnover_rate,volume_ratio,pe,pb,total_share,free_share,total_mv,circ_mv,...,q_dt_roe,capital_rese_ps,surplus_rese_ps,gross_margin,interestdebt,ca_to_assets,ebt_yoy,roe_yoy,or_yoy,equity_yoy
0,000028.SZ,20230327,1.4743,0.79,13.9609,1.157,42812.6983,12791.6361,1.865777e+06,1.602583e+06,...,2.2888,10.2115,0.5,6.273907e+09,5.167881e+09,76.9456,-1.8966,-8.9,8.7766,7.0603
1,000028.SZ,20230324,1.9247,0.75,14.2236,1.1788,42812.6983,12791.6361,1.900884e+06,1.632737e+06,...,2.2888,10.2115,0.5,6.273907e+09,5.167881e+09,76.9456,-1.8966,-8.9,8.7766,7.0603
2,000028.SZ,20230323,1.0107,0.33,14.5792,1.2082,42812.6983,12791.6361,1.948406e+06,1.673556e+06,...,2.2888,10.2115,0.5,6.273907e+09,5.167881e+09,76.9456,-1.8966,-8.9,8.7766,7.0603
3,000028.SZ,20230322,1.5294,0.51,14.576,1.208,42812.6983,12791.6361,1.947978e+06,1.673188e+06,...,2.2888,10.2115,0.5,6.273907e+09,5.167881e+09,76.9456,-1.8966,-8.9,8.7766,7.0603
4,000028.SZ,20230321,1.6397,0.55,14.6401,1.2133,42812.6983,12791.6361,1.956540e+06,1.680543e+06,...,2.2888,10.2115,0.5,6.273907e+09,5.167881e+09,76.9456,-1.8966,-8.9,8.7766,7.0603


In [81]:
# merage all
universe = universe_raw.copy(deep=True)
universe = universe.drop_duplicates(['trade_date','ts_code'])
universe = universe.merge(finance_df_all, on=['ts_code','trade_date'], how='left')
# fillna use ffill then fillna(0)
universe_raw = pd.DataFrame()
for group_ticker in tqdm(universe.groupby('ts_code'), desc='aggregeate df'):
    ticker = group_ticker[1]
    ticker.fillna(method='ffill', inplace=True)
    universe_raw = universe_raw.append(ticker)
universe_raw.reset_index(drop=True, inplace=True)

aggregeate df: 100%|█████████████████████████| 650/650 [00:05<00:00, 113.50it/s]


In [82]:
view_columns = ['trade_date','ts_code','name','open','close','pct_change','amount','volume_ratio','pe',
                'type','p_change_min','p_change_max','list_date']
#universe_raw.loc[universe_raw.ts_code=='603538.SH'][view_columns]
print(universe_raw.shape)
universe_raw.loc[universe_raw.ts_code=='603538.SH'].tail()

(35585, 47)


,ts_code,trade_date,turnover_rate,volume_ratio,pe,pb,total_share,free_share,total_mv,circ_mv,...,roe_yoy,capital_rese_ps,surplus_rese_ps,gross_margin,interestdebt,ca_to_assets,ebt_yoy,roe_yoy,or_yoy,equity_yoy
28225,603538.SH,20230109,3.0649,0.92,38.6105,2.9164,21337.7812,15368.0025,550301.3771,544261.6867,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28226,603538.SH,20230106,2.6777,0.75,38.7602,2.9277,21337.7812,15368.0025,552435.1553,546372.0461,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28227,603538.SH,20230105,3.5007,0.88,39.0596,2.9503,21337.7812,15368.0025,556702.7115,550592.7649,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28228,603538.SH,20230104,3.8971,0.70,38.5356,2.9108,21337.7812,15321.6878,549234.4881,542014.3666,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28229,603538.SH,20230103,3.3397,0.54,39.2692,2.9662,21337.7812,15321.6878,559690.0009,552332.4334,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Save Raw Data

In [45]:
#'20170103', '20230317'
universe_raw = universe_raw.drop_duplicates(keep='first').reset_index(drop=True)
fundamental_df = fundamental_df.drop_duplicates(keep='first').reset_index(drop=True)
universe_raw.to_csv('raw_20230103_20230327.csv')
#fundamental_df.to_csv('fundamental_20230103_20230327.csv')

## Load Data

In [33]:
universe_raw = pd.read_csv('raw_20170103_20230317.csv').iloc[:,1:]
fundamental_df = pd.read_csv('fundamental_20170103_20230317.csv').iloc[:,1:]